In [74]:
import numpy as np
import os
os.environ['PROJ_LIB'] = r'C:\Users\anany\AppData\Local\Continuum\anaconda3\pkgs\proj4-5.2.0-ha925a31_1\Library\share'
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap
import pandas as pd
from pandas import DataFrame, Series
import re
import itertools
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
import statsmodels.api as sm
from pylab import rcParams
from statsmodels.tsa.seasonal import seasonal_decompose

%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [66]:
df = pd.read_csv("Fire_Data.csv")
df = df.dropna()
df = df[['Date', 'Geolocation']]
def getLocation(val):
    l = re.findall('\[.*\]', val)[0]
    l = l.lstrip('[').rstrip(']')
    l = l.split(',')
    return [float(l[0]), float(l[1])]
def getX(val):
    return val[0]
def getY(val):
    return val[1]
df['loc'] = df['Geolocation'].apply(getLocation)
df['x'] = df['loc'].apply(getX)
df['y'] = df['loc'].apply(getY)
df = df[['Date', 'x', 'y']]

In [110]:
df = pd.read_csv("Fire_Data.csv")
df = df.dropna()
df = df[['Date', 'Geolocation']]
def getLocation(val):
    l = re.findall('\[.*\]', val)[0]
    l = l.lstrip('[').rstrip(']')
    l = l.split(',')
    return [float(l[0]), float(l[1])]
def getX(val):
    return val[0]
def getY(val):
    return val[1]
df['loc'] = df['Geolocation'].apply(getLocation)
df['x'] = df['loc'].apply(getX)
df['y'] = df['loc'].apply(getY)
df = df[['Date', 'x', 'y']]
##################################################################################
df = df.sort_values(by = 'Date')
df['Date'] = pd.to_datetime(df['Date'])
df = df.set_index('Date')

y = df['x'].resample('12H').mean().to_frame().reset_index()
y['ffilData'] = y['x'].fillna(method='ffill')
y['backfillData'] = y['x'].fillna(method='bfill')
y['x'] = (y['ffilData'] + y['backfillData'])/2

y = y[['Date', 'x']]
y = y.set_index('Date')

result_mul = seasonal_decompose(y['x'], model='additive', extrapolate_trend='freq', freq = 3)
listOfArima = []
p = d = q = range(0, 2)
pdq = list(itertools.product(p, d, q))
seasonal_pdq = [(x[0], x[1], x[2], 12) for x in list(itertools.product(p, d, q))]
for param in pdq:
    for param_seasonal in seasonal_pdq:
        try:
            mod = sm.tsa.statespace.SARIMAX(y, order=param, 
                                            seasonal_order=param_seasonal, 
                                            enforce_stationarity=False, 
                                            enforce_invertibility=False)
            results = mod.fit()
            listOfArima.append([param, param_seasonal, results.aic])
        except:
            continue
min = 1000
j = 0
for i in listOfArima:
    if i[2] < min:
        j = i
        min = i[2]
mod = sm.tsa.statespace.SARIMAX(y,
                                order=j[0],
                                seasonal_order=j[1],
                                enforce_stationarity=False,
                                enforce_invertibility=False)
results = mod.fit()
j = results.forecast(steps = 4).to_frame()
j_x = j.rename(columns = {0:'x'})


In [111]:
y = df['y'].resample('12H').mean().to_frame().reset_index()
y['ffilData'] = y['y'].fillna(method='ffill')
y['backfillData'] = y['y'].fillna(method='bfill')
y['y'] = (y['ffilData'] + y['backfillData'])/2
y = y[['Date', 'y']]
y = y.set_index('Date')

result_mul = seasonal_decompose(y['y'], model='additive', extrapolate_trend='freq', freq = 3)
listOfArima = []
p = d = q = range(0, 2)
pdq = list(itertools.product(p, d, q))
seasonal_pdq = [(x[0], x[1], x[2], 12) for x in list(itertools.product(p, d, q))]
for param in pdq:
    for param_seasonal in seasonal_pdq:
        try:
            mod = sm.tsa.statespace.SARIMAX(y, order=param, 
                                            seasonal_order=param_seasonal, 
                                            enforce_stationarity=False, 
                                            enforce_invertibility=False)
            results = mod.fit()
            listOfArima.append([param, param_seasonal, results.aic])
        except:
            continue
min = 1000
j = 0
for i in listOfArima:
    if i[2] < min:
        j = i
        min = i[2]
mod = sm.tsa.statespace.SARIMAX(y,
                                order=j[0],
                                seasonal_order=j[1],
                                enforce_stationarity=False,
                                enforce_invertibility=False)
results_y = mod.fit()

j = results.forecast(steps = 4).to_frame()
j_y = j.rename(columns = {0:'y'})


In [112]:
J_x_1 = j_x.reset_index()
J_x_1 = J_x_1.rename(columns = {'index':'Date'})

J_y_1 = j_y.reset_index()
J_y_1 = J_y_1.rename(columns = {'index':'Date'})

dfnew = J_x_1.merge(J_y_1, left_on = 'Date', right_on = 'Date')
dfnew['Date'] = dfnew.index + 1

def toStringData(val):
    return str(val)

dfnew['x'] = dfnew['x'].map(toStringData)
dfnew['y'] = dfnew['y'].map(toStringData)
dfnew = dfnew[['y', 'x', 'Date']]
dfnew = dfnew.values.tolist()
print(dfnew)

[['34.36930492706232', '-119.10185276526958', 1], ['34.368365960097414', '-119.135715155373', 2], ['34.36631810673867', '-119.16958717304676', 3], ['34.36915429839143', '-119.2034688210281', 4]]


In [46]:
# pred_uc = results.get_forecast(steps=12)
# pred_ci = pred_uc.conf_int()

# ax = y.plot(label='observed', figsize=(14, 7))
# pred_uc.predicted_mean.plot(ax=ax, label='Forecast')
# ax.fill_between(pred_ci.index,
#                 pred_ci.iloc[:, 0],
#                 pred_ci.iloc[:, 1], color='k', alpha=.25)
# ax.set_xlabel('Date')
# ax.set_ylabel('Furniture Sales')
# plt.legend()
# plt.show()

In [41]:

# fig = plt.figure(figsize=(20, 20))
# meanX = df['x'].mean()
# meanY = df['y'].mean()
# print(meanX)
# print(meanY)
# m = Basemap(projection='lcc', resolution='h',
#             width=1E5, height=1.2E5,
#             llcrnrlon = meanX - 1,
#             llcrnrlat = meanY - 1, 
#             urcrnrlon = meanX + 1, 
#             urcrnrlat = meanY + 1,
#            lat_0=meanY, lon_0=meanX, epsg = 4326)
# m.etopo()
# type_of_service = 'NatGeo_World_Map'
# m.arcgisimage(service = type_of_service, xpixels = 2000)

# for i in range(len(df)):
#     x, y = m(df.iloc[i, 1], df.iloc[i, 2])
#     plt.plot(x, y, 'ok', markersize=5)
#     plt.text(x, y, df.iloc[i, 0], fontsize=12);


In [104]:
df = pd.read_csv("final_output_new.csv")
print(df.shape)

(19, 4)
